## Interactive Dashboard of COVID-19 Data

In this section we created an **interactive dashboard** using the Dash framework.  
The application allows users to select a country and view cumulative charts of the pandemic.

### Key steps:
- Filtered the dataset to keep only real countries.
- Forward-filled cumulative values of cases and deaths for each country.
- Prepared a list of available countries for the dropdown menu.

In [2]:
# --- Imports ---
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

# --- Load data ---
file_path = "../Data/Covid_19_dataset.csv"
df = pd.read_csv(file_path)

# Ensure datetime
df["date"] = pd.to_datetime(df["date"], errors="coerce")

# --- Project palette (consistent with previous notebooks) ---
project_palette = ["#0474ed","#91f0fa","#08a29e","#a2b458","#a6cabd","#326164"]

# for Plotly Express figures in Dash
palette_dash = project_palette

In [6]:
# --- Clean & prepare ---
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df = df[~df["iso_code"].str.startswith("OWID_")].copy()        # keep real countries only
df = df.sort_values(["location", "date"])
df[["total_cases","total_deaths"]] = df.groupby("location")[["total_cases","total_deaths"]].ffill()

countries = sorted(df["location"].dropna().unique().tolist())
default_country = "Czechia" if "Czechia" in countries else countries[0]

# --- Palette (same project colors) ---
palette_dash = ["#0474ed","#91f0fa","#08a29e","#a2b458","#a6cabd","#326164"]
cases_color, deaths_color = palette_dash[0], palette_dash[3]

# --- App ---
app = Dash(__name__)
app.title = "COVID-19 — Cumulative Charts"

APP_WIDTH = 1200

app.layout = html.Div(
    style={
        "fontFamily": "Arial, sans-serif",
        "background": "#fff",          # celé bílé
        "padding": "16px",
        "width": f"{APP_WIDTH}px",
        "margin": "0 auto"             # vycentrovat v notebooku
    },
    children=[
        html.H2(id="page-title",
                style={"marginBottom":"8px","color":"#111"}),

        html.Div(
            [
                html.Label("Select country:",
                           style={"fontWeight":"600","marginRight":"8px","color":"#111"}),
                dcc.Dropdown(
                    id="country-dropdown",
                    options=[{"label": c, "value": c} for c in countries],
                    value=default_country,
                    clearable=False,
                    style={"width":"420px"}
                )
            ],
            style={"marginBottom":"12px"}
        ),

        html.Div(
            [
                dcc.Graph(id="cases-chart"),
                dcc.Graph(id="deaths-chart"),
            ],
            style={"display":"grid","gridTemplateColumns":"1fr 1fr","gap":"16px"}
        ),
    ]
)

# --- Callbacks ---
@app.callback(
    Output("page-title", "children"),
    Output("cases-chart", "figure"),
    Output("deaths-chart", "figure"),
    Input("country-dropdown", "value")
)
def update_dashboard(country):
    d = df[df["location"] == country].sort_values("date")[["date","total_cases","total_deaths"]].dropna(subset=["date"])

    title = f"Cumulative cases and deaths in {country}"

    fig_cases = px.line(
        d, x="date", y="total_cases",
        labels={"date":"Date","total_cases":"Cumulative Cases"},
        title=f"Cumulative Cases — {country}",
    )
    fig_cases.update_traces(line=dict(width=2, color=cases_color))
    fig_cases.update_layout(margin=dict(l=10,r=10,t=50,b=10), height=420)

    fig_deaths = px.line(
        d, x="date", y="total_deaths",
        labels={"date":"Date","total_deaths":"Cumulative Deaths"},
        title=f"Cumulative Deaths — {country}",
    )
    fig_deaths.update_traces(line=dict(width=2, color=deaths_color))
    fig_deaths.update_layout(margin=dict(l=10,r=10,t=50,b=10), height=420)

    return title, fig_cases, fig_deaths

## Explanation

**Cumulative cases** represent the total number of confirmed infections up to each date.  
**Cumulative deaths** represent the total number of reported fatalities up to each date.  

Unlike daily numbers, cumulative values never decrease — they either grow or stay flat.  
This makes it easier to compare the overall scale and timing of the pandemic across countries.

In [7]:
app.run(debug=True, use_reloader=False)

![Deaths](../Assets/graph5_deaths.png)
![Deaths trend](../Assets/graph6_deaths.png)

## Summary

The dashboard shows that cumulative cases and cumulative deaths in each country  
follow very similar patterns over time. This strong resemblance is unlikely to be a coincidence,  
but rather reflects the close relationship between the spread of infections and the resulting fatalities.